<a href="https://colab.research.google.com/github/Tandon-A/SPAIC-Udacity/blob/master/FederatedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install syft 

     |████████████████████████████████| 317kB 3.5MB/s 
     |████████████████████████████████| 256kB 42.3MB/s 
     |████████████████████████████████| 81kB 29.1MB/s 
     |████████████████████████████████| 460kB 48.2MB/s 
     |████████████████████████████████| 389kB 29.3MB/s 
     |████████████████████████████████| 2.1MB 35.5MB/s 
     |████████████████████████████████| 204kB 43.9MB/s 
     |████████████████████████████████| 51kB 21.9MB/s 
     |████████████████████████████████| 266kB 49.8MB/s 
     |████████████████████████████████| 122kB 50.1MB/s 
  Created wheel for zstd: filename=zstd-1.4.3.2-cp36-cp36m-linux_x86_64.whl size=1076248 sha256=fbdadca6ce5f8ebd546b03009f1709c83015ebb811793e76f01e82d07edb350e
  Stored in directory: /root/.cache/pip/wheels/f2/df/a8/405bafcabba88a18c8763694e79177e2a1bbc65ac0f6b3d728
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=7a01fa9dabc56440e5145064d61a2ac9086f06ba9edd0a09e746ab8d73c278f1
  Stored in di

In [2]:
from torchvision import datasets, transforms
import numpy as np
import torch.nn as nn
from torch import optim
from collections import OrderedDict
import torch 


#Loading MNIST data 
mnist = datasets.MNIST(root='./data', train=True, download=True, transform= transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
trainX = mnist.train_data
trainY = mnist.train_labels
np.random.seed(7) 

#shuffling MNIST data
perm = np.random.permutation(len(trainX))
trainX = trainX[perm,:]
trainY = trainY[perm]

#Dividing the dataset into three training sets and a validation set 
val_x = trainX[40000:50000]
val_y = trainY[40000:50000]
tr_x1 = trainX[0:10000]
tr_y1 = trainY[0:10000]
tr_x2 = trainX[10000:20000]
tr_y2 = trainY[10000:20000]
tr_x3 = trainX[20000:30000]
tr_y3 = trainY[20000:30000]

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20296495.84it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 306205.14it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5359133.32it/s]                           
8192it [00:00, 126253.41it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


Done!


In [0]:
#function for creating a pytorch model
def create_model():
  return nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(784,512)),
    ('r1',nn.ReLU()),
    ('dropout1',nn.Dropout(p=0.2)),
    ('fc2',nn.Linear(512,512)),
    ('r2',nn.ReLU()),
    ('dropout2',nn.Dropout(p=0.2)),
    ('fc3',nn.Linear(512,256)),
    ('r3',nn.ReLU()), 
    ('fc4',nn.Linear(256,64)),
    ('r4',nn.ReLU()),
    ('fc5',nn.Linear(64,10)),
    ('log_ps',nn.LogSoftmax(dim=1))]))



criterion = nn.NLLLoss()

In [4]:
import syft as sy 

#Creating a torch hook for PySyft
hook = sy.TorchHook(torch)
#Creating three virtual workers for federated learning 
bob = sy.VirtualWorker(hook,id='bob')
alice = sy.VirtualWorker(hook,id='alice')
arya = sy.VirtualWorker(hook,id='arya')

bob.clear_objects()
alice.clear_objects()
arya.clear_objects()
print (bob._objects,alice._objects,arya._objects)

{} {} {}


In [5]:
workers = [bob,alice,arya]
datasets = [(tr_x1,tr_y1),(tr_x2,tr_y2),(tr_x3,tr_y3)]
#list for saving the models trained on different workers
models = list()



for i in range(len(workers)):
  #sending validation data to specific worker
  val_x_worker = val_x.clone().send(workers[i])
  val_y_worker = val_y.clone().send(workers[i])
  tr_x,tr_y = datasets[i]
  #sending dataset to worker
  tr_x = tr_x.send(workers[i])
  tr_y = tr_y.send(workers[i])
  #creating a model for a worker
  model_worker = create_model()
  #creating optimizer for worker -- right now pysyft only allows for SGD as optimizer
  opt = optim.SGD(model_worker.parameters(),lr=0.001)
  model_worker.send(workers[i])
  
  
  #training worker models for 5 epochs 
  epochs = 5
  batch_size = 50
  batches = len(tr_x)//batch_size
  #setting worker model in train mode
  model_worker.train()
  print ("starting training for model ",i)
  for e in range(epochs):
    running_loss = 0
    for batch in range(batches):
      opt.zero_grad()
      images = tr_x[batch*batch_size : (batch+1)*batch_size].float().view(batch_size,-1)
      labels = tr_y[batch*batch_size : (batch+1)*batch_size]
      log_ps = model_worker(images)
      loss = criterion(log_ps,labels)
      running_loss += loss
      loss.backward()
      opt.step()
    running_loss = running_loss.get()
    #print (running_loss)
  #testing worker model on validation data  
  with torch.no_grad():
    model_worker.eval()
    ps = torch.exp(model_worker(val_x_worker.view(val_x_worker.shape[0],-1).float()))
    topp,topk = ps.topk(1,dim=1)
    equals = topk == val_y_worker.view(*topk.shape)
    equals = equals.get()
    accuracy = torch.mean(equals.type(torch.FloatTensor)) 
    print ("worker model = %r accuracy = %r " %(i,accuracy))
  #saving the worker model  
  models.append(model_worker)

starting training for model  0
worker model = 0 accuracy = tensor(0.9184) 
starting training for model  1
worker model = 1 accuracy = tensor(0.9191) 
starting training for model  2
worker model = 2 accuracy = tensor(0.9139) 


In [6]:
model_arya = models[-1]
model_alice = models[-2]
model_bob = models[-3]
print (model_arya.location,model_alice.location,model_bob.location)
#shifting all models to bob worker for aggregation
model_arya.move(bob)
model_alice.move(bob)
print (model_arya.location,model_alice.location,model_bob.location)

<VirtualWorker id:arya #objects:26> <VirtualWorker id:alice #objects:15> <VirtualWorker id:bob #objects:15>
<VirtualWorker id:bob #objects:35> <VirtualWorker id:bob #objects:35> <VirtualWorker id:bob #objects:35>


In [0]:
final_model = create_model().send(bob)
bobp = model_bob.parameters()
alicep = model_alice.parameters()
aryap = model_arya.parameters()
finalp = final_model.parameters()

#the final model now stores the aggregation of all the trained models 
with torch.no_grad():
  for pf,pb,pa,par in zip(finalp,bobp,alicep,aryap):
    pf.set_((pb + pa + par)/3)